In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('video-game-sales.csv')
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [4]:
df.isnull().mean()*100

Name                0.011962
Platform            0.000000
Year_of_Release     1.608948
Genre               0.011962
Publisher           0.322986
NA_Sales            0.000000
EU_Sales            0.000000
JP_Sales            0.000000
Other_Sales         0.000000
Global_Sales        0.000000
Critic_Score       51.330821
Critic_Count       51.330821
User_Score         40.098092
User_Count         54.602548
Developer          39.613613
Rating             40.486871
dtype: float64

In [5]:
df['User_Score'].value_counts()

tbd    2425
7.8     324
8       290
8.2     282
8.3     254
8.5     253
7.5     251
7.9     249
8.1     244
7.7     240
7.3     236
8.4     223
7       220
7.6     220
7.4     215
8.6     211
6.8     197
8.7     188
8.8     186
7.1     180
7.2     167
8.9     153
6.6     148
6.9     143
6.3     138
6.7     128
6       127
6.5     125
9       120
6.2     113
6.4     107
5.8      97
9.1      90
6.1      84
5.7      78
5.9      77
5.6      72
5.4      72
5.5      71
5.3      70
5        64
5.2      57
4.8      49
5.1      44
9.2      43
4.9      40
4.6      37
4.3      37
4.5      34
4.4      33
4.1      33
9.3      31
4.2      28
3.8      28
4        27
3.5      26
2.8      24
4.7      24
3.4      23
3.1      23
3        21
3.6      20
3.7      19
3.3      15
3.9      13
3.2      13
2.5      12
2.4      11
2        11
9.4      11
2.1       9
1.7       9
2.9       9
2.7       8
9.5       6
1.8       6
2.2       6
1.4       5
2.6       4
1.6       3
1.2       3
0.5       2
1         2
0.9 

In [6]:
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       7590 non-null   float64
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(10), object(6)
memory usage: 2.0+ MB


In [9]:
df.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         9129
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [14]:
df['Name'].value_counts()

Need for Speed: Most Wanted                         12
FIFA 14                                              9
Ratatouille                                          9
LEGO Marvel Super Heroes                             9
Madden NFL 07                                        9
                                                    ..
Jewels of the Tropical Lost Island                   1
Sherlock Holmes and the Mystery of Osborne House     1
The King of Fighters '95 (CD)                        1
Megamind: Mega Team Unite                            1
Haitaka no Psychedelica                              1
Name: Name, Length: 11562, dtype: int64

In [17]:
df = df[df['Name'].notna()]
df.isnull().sum()

Name                  0
Platform              0
Year_of_Release     269
Genre                 0
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8580
Critic_Count       8580
User_Score         9127
User_Count         9127
Developer          6621
Rating             6767
dtype: int64

In [20]:
# Separate numeric and categorical columns
df_numeric = df.select_dtypes(np.number)
df_categorical = df.select_dtypes(object)

# Impute null values with mean and mode
df_num_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df_cat_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_num_imp = df_num_imputer.fit_transform(df_numeric)
df_cat_imp = df_cat_imputer.fit_transform(df_categorical)

# Transform imputed data from numpy array to dataframe
num_imp_df = pd.DataFrame(df_num_imp, columns=df_numeric.columns)
cat_imp_df = pd.DataFrame(df_cat_imp, columns=df_categorical.columns)

# Combine DataFrames
df_cleaned = pd.concat([num_imp_df, cat_imp_df], axis = 1)

df_cleaned.head()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Name,Platform,Genre,Publisher,Developer,Rating
0,2006.0,41.36,28.96,3.77,8.45,82.53,76.000000,51.000000,8.000000,322.000000,Wii Sports,Wii,Sports,Nintendo,Nintendo,E
1,1985.0,29.08,3.58,6.81,0.77,40.24,68.967679,26.360821,7.125046,162.229908,Super Mario Bros.,NES,Platform,Nintendo,Ubisoft,E
2,2008.0,15.68,12.76,3.79,3.29,35.52,82.000000,73.000000,8.300000,709.000000,Mario Kart Wii,Wii,Racing,Nintendo,Nintendo,E
3,2009.0,15.61,10.93,3.28,2.95,32.77,80.000000,73.000000,8.000000,192.000000,Wii Sports Resort,Wii,Sports,Nintendo,Nintendo,E
4,1996.0,11.27,8.89,10.22,1.00,31.37,68.967679,26.360821,7.125046,162.229908,Pokemon Red/Pokemon Blue,GB,Role-Playing,Nintendo,Ubisoft,E


In [21]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16717 entries, 0 to 16716
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year_of_Release  16717 non-null  float64
 1   NA_Sales         16717 non-null  float64
 2   EU_Sales         16717 non-null  float64
 3   JP_Sales         16717 non-null  float64
 4   Other_Sales      16717 non-null  float64
 5   Global_Sales     16717 non-null  float64
 6   Critic_Score     16717 non-null  float64
 7   Critic_Count     16717 non-null  float64
 8   User_Score       16717 non-null  float64
 9   User_Count       16717 non-null  float64
 10  Name             16717 non-null  object 
 11  Platform         16717 non-null  object 
 12  Genre            16717 non-null  object 
 13  Publisher        16717 non-null  object 
 14  Developer        16717 non-null  object 
 15  Rating           16717 non-null  object 
dtypes: float64(10), object(6)
memory usage: 2.0+ MB


### Base Model XGBoost

In [37]:
X = df_cleaned.drop(columns=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales'])
y = df_cleaned['Global_Sales']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [39]:
pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ('sc', StandardScaler()),
    ('xgb', XGBRegressor())
])

In [40]:
%%time
pipe.fit(X_train, y_train)

CPU times: user 23min 8s, sys: 31.5 s, total: 23min 40s
Wall time: 10min 36s


Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('sc', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

In [41]:
pipe.score(X_train, y_train)

0.9459737158316267

In [42]:
pipe.score(X_test, y_test)

0.46631870789684193

In [43]:
preds = pipe.predict(X_test)
mean_squared_error(y_test, preds)

1.6899883714953126

In [44]:
mean_squared_error(y_test, preds, squared=False)

1.2999955274905035

### Model Tuning XGBoost

In [45]:
X = df_cleaned.drop(columns=['NA_Sales', 'EU_Sales', 'Global_Sales'])
y = df_cleaned['Global_Sales']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [47]:
pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ('sc', StandardScaler()),
    ('xgb', XGBRegressor())
])

In [48]:
%%time
pipe.fit(X_train, y_train)

KeyboardInterrupt: 

In [49]:
pipe.score(X_train, y_train)

NotFittedError: need to call fit or load_model beforehand

In [ ]:
pipe.score(X_test, y_test)

In [ ]:
preds = pipe.predict(X_test)
mean_squared_error(y_test, preds)

In [ ]:
mean_squared_error(y_test, preds, squared=False)